Source:
https://robust-dinosaur-2ef.notion.site/DuckDB-Tutorial-Getting-started-for-beginners-b80bf0de8d6142d6979e78e59ffbbefe

Video: https://www.youtube.com/watch?v=AjsB6lM2-zw

In [5]:
import pandas as pd
import glob
import time
import duckdb

conn = duckdb.connect() # create an in-memory database

In [6]:
# Print the first 10 rows
# with pandas
cur_time = time.time()
df = pd.concat([pd.read_csv(f) for f in glob.glob('dataset/*.csv')])
print(f"time: {(time.time() - cur_time)}")
print(df.head(10))

time: 0.3006303310394287
   Order ID                     Product  Quantity Ordered     Price  \
0    176558        USB-C Charging Cable                 2     11.95   
1    176559  Bose SoundSport Headphones                 1     99.99   
2    176560                Google Phone                 1       600   
3    176560            Wired Headphones                 1     11.99   
4    176561            Wired Headphones                 1     11.99   
5    176562        USB-C Charging Cable                 1     11.95   
6    176563  Bose SoundSport Headphones                 1     99.99   
7    176564        USB-C Charging Cable                 1     11.95   
8    176565          Macbook Pro Laptop                 1  1,700.00   
9    176566            Wired Headphones                 1     11.99   

   Order Date      Time                        Purchase Address  \
0  19-04-2019   8:46 AM            917 1st St, Dallas, TX 75001   
1  07-04-2019  10:30 PM       682 Chestnut St, Boston, MA 0

In [7]:
# Print the first 10 rows
# with duckdb
cur_time = time.time()
df = conn.execute("""
	SELECT *
	FROM 'dataset/*.csv'
	LIMIT 10
""").df()
print(f"time: {(time.time() - cur_time)}")
print(df)

time: 0.08657479286193848
   Order ID                     Product  Quantity Ordered     Price  \
0    176558        USB-C Charging Cable                 2     11.95   
1    176559  Bose SoundSport Headphones                 1     99.99   
2    176560                Google Phone                 1       600   
3    176560            Wired Headphones                 1     11.99   
4    176561            Wired Headphones                 1     11.99   
5    176562        USB-C Charging Cable                 1     11.95   
6    176563  Bose SoundSport Headphones                 1     99.99   
7    176564        USB-C Charging Cable                 1     11.95   
8    176565          Macbook Pro Laptop                 1  1,700.00   
9    176566            Wired Headphones                 1     11.99   

  Order Date      Time                        Purchase Address  \
0 2019-04-19   8:46 AM            917 1st St, Dallas, TX 75001   
1 2019-04-07  10:30 PM       682 Chestnut St, Boston, MA 022

In [8]:
# Checking the types of the columns
df = conn.execute("""
	SELECT *
	FROM 'dataset/*.csv'
""").df()
conn.register("df_view", df)
conn.execute("DESCRIBE df_view").df() # doesn't work if you don't register df as a virtual table

,column_name,column_type,null,key,default,extra
0,Order ID,BIGINT,YES,None,None,None
1,Product,VARCHAR,YES,None,None,None
2,Quantity Ordered,BIGINT,YES,None,None,None
3,Price,VARCHAR,YES,None,None,None
4,Order Date,TIMESTAMP,YES,None,None,None
5,Time,VARCHAR,YES,None,None,None
6,Purchase Address,VARCHAR,YES,None,None,None
7,City,VARCHAR,YES,None,None,None
8,Product Type,VARCHAR,YES,None,None,None


Counting rows

In [9]:
conn.execute("SELECT COUNT(*) FROM df_view").df()

,count_star()
0,185950


Drop nulls

In [10]:
df.isnull().sum()
df = df.dropna(how='all')

# Notice we use df and not df_view
# With DuckDB you can run SQL queries on top of Pandas dataframes
conn.execute("SELECT COUNT(*) FROM df").df()

,count_star()
0,185950


Where clause

"Make sure to use 3 double quotes at the beginning and the end of your statement + use doubles quotes if your column is not a single word colum"

In [11]:
conn.execute("""SELECT * FROM df WHERE "Order ID"='295665'""").df()

,Order ID,Product,Quantity Ordered,Price,Order Date,Time,Purchase Address,City,Product Type
0,295665,Macbook Pro Laptop,1,"1,700.00",2019-12-30,12:01 AM,"136 Church St, New York City, NY 10001",New York City,Laptop


Create a table and load the data:
A View/Virtual Table is a SELECT statement. That statement is run every time the view is referenced in a query. Views are great for abstracting the complexity of the underlying tables they reference.

TRY_CAST returns null if the cast fails. By using TRY_CAST NOTNULL we skip malformatted rows. 

In [21]:
conn.execute("""
CREATE OR REPLACE TABLE sales AS
    SELECT
        TRY_CAST("Order ID" AS INTEGER) AS order_id,
        Product AS product,
        TRY_CAST("Quantity Ordered" AS INTEGER) AS quantity,
        TRY_CAST("Price" AS DECIMAL(18,3)) AS price_each,
        "Order Date" AS order_date,
        "Purchase Address" AS purchase_address
    FROM df
    WHERE TRY_CAST("Order ID" AS INTEGER) IS NOT NULL
""")


In [22]:
conn.execute("FROM sales").df()

,order_id,product,quantity,price_each,order_date,purchase_address
0,176558,USB-C Charging Cable,2,11.95,2019-04-19,"917 1st St, Dallas, TX 75001"
1,176559,Bose SoundSport Headphones,1,99.99,2019-04-07,"682 Chestnut St, Boston, MA 02215"
2,176560,Google Phone,1,600.00,2019-04-12,"669 Spruce St, Los Angeles, CA 90001"
3,176560,Wired Headphones,1,11.99,2019-04-12,"669 Spruce St, Los Angeles, CA 90001"
4,176561,Wired Headphones,1,11.99,2019-04-30,"333 8th St, Los Angeles, CA 90001"
...,...,...,...,...,...,...
185945,259353,AAA Batteries (4-pack),3,2.99,2019-09-17,"840 Highland St, Los Angeles, CA 90001"
185946,259354,iPhone,1,700.00,2019-09-01,"216 Dogwood St, San Francisco, CA 94016"
185947,259355,iPhone,1,700.00,2019-09-23,"220 12th St, San Francisco, CA 94016"
185948,259356,34in Ultrawide Monitor,1,379.99,2019-09-19,"511 Forest St, San Francisco, CA 94016"


In [27]:
# FROM-first clause
conn.execute("FROM sales").df()

,order_id,product,quantity,price_each,order_date,purchase_address
0,176558,USB-C Charging Cable,2,11.95,2019-04-19,"917 1st St, Dallas, TX 75001"
1,176559,Bose SoundSport Headphones,1,99.99,2019-04-07,"682 Chestnut St, Boston, MA 02215"
2,176560,Google Phone,1,600.00,2019-04-12,"669 Spruce St, Los Angeles, CA 90001"
3,176560,Wired Headphones,1,11.99,2019-04-12,"669 Spruce St, Los Angeles, CA 90001"
4,176561,Wired Headphones,1,11.99,2019-04-30,"333 8th St, Los Angeles, CA 90001"
...,...,...,...,...,...,...
185945,259353,AAA Batteries (4-pack),3,2.99,2019-09-17,"840 Highland St, Los Angeles, CA 90001"
185946,259354,iPhone,1,700.00,2019-09-01,"216 Dogwood St, San Francisco, CA 94016"
185947,259355,iPhone,1,700.00,2019-09-23,"220 12th St, San Francisco, CA 94016"
185948,259356,34in Ultrawide Monitor,1,379.99,2019-09-19,"511 Forest St, San Francisco, CA 94016"


In [32]:
# Exclude
conn.execute("""
	SELECT 
		* EXCLUDE (product, order_date, purchase_address)
	FROM sales
	""").df()

,order_id,quantity,price_each
0,176558,2,11.95
1,176559,1,99.99
2,176560,1,600.00
3,176560,1,11.99
4,176561,1,11.99
...,...,...,...
185945,259353,3,2.99
185946,259354,1,700.00
185947,259355,1,700.00
185948,259356,1,379.99


In [31]:
#The Columns Expression
conn.execute("""
	SELECT 
		MIN(COLUMNS(* EXCLUDE (product, order_date, purchase_address))) 
	FROM sales
	""").df()

,order_id,quantity,price_each
0,141234,1,2.99


In [33]:
# Create a VIEW
conn.execute("""
	CREATE OR REPLACE VIEW aggregated_sales AS
	SELECT
		order_id,
		COUNT(1) as nb_orders,
		MONTH(order_date) as month,
		str_split(purchase_address, ',')[2] AS city,
		SUM(quantity * price_each) AS revenue
	FROM sales
	GROUP BY ALL
""")

In [35]:
#Export to Parquet files
conn.execute("COPY (FROM aggregated_sales) TO 'aggregated_sales.parquet' (FORMAT 'parquet')")

In [36]:
#Query Parquet files

conn.execute("FROM aggregated_sales.parquet").df()

,order_id,nb_orders,month,city,revenue
0,196952,1,5,Los Angeles,11.95
1,196962,1,5,Austin,14.95
2,196963,1,5,New York City,NaN
3,196984,1,5,San Francisco,3.84
4,197004,2,5,New York City,113.83
...,...,...,...,...,...
178432,196863,1,5,San Francisco,149.99
178433,196885,1,5,New York City,3.84
178434,196895,1,5,Boston,23.90
178435,196902,1,5,San Francisco,14.95
